In [1]:
import argparse
import scipy
from scipy import ndimage
import numpy as np
import sys

import torch
from torch.autograd import Variable
import torchvision.models as models
import torch.nn.functional as F
from torch.utils import data, model_zoo
from model.deeplab_single import Res_Deeplab
from dataset.cityscapes_dataset import cityscapesDataSet
from collections import OrderedDict
import os
from PIL import Image

import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)

DATA_DIRECTORY = './data/Cityscapes/data'
DATA_LIST_PATH = './dataset/cityscapes_list/val.txt'
SAVE_PATH = './result/cityscapes'

IGNORE_LABEL = 255
NUM_CLASSES = 19
NUM_STEPS = 500 # Number of images in the validation set.
RESTORE_FROM = 'http://vllab.ucmerced.edu/ytsai/CVPR18/GTA2Cityscapes_multi-ed35151c.pth'
SET = 'val'

palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)
for i in range(zero_pad):
    palette.append(0)


def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask


In [3]:
gpu0 = 0
num_classes = 19
restore_from = "./snapshots/discriminator_more/GTA5_10000.pth"

In [4]:
model = Res_Deeplab(num_classes=num_classes)

In [5]:
saved_state_dict = torch.load(restore_from)
model.load_state_dict(saved_state_dict)


In [6]:
model.eval()
model.cuda(gpu0)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [7]:
DATA_DIRECTORY = './data/Cityscapes/data'
DATA_LIST_PATH = './dataset/cityscapes_list/val.txt'
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)
SET = 'val'

In [8]:
testloader = data.DataLoader(cityscapesDataSet(DATA_DIRECTORY, DATA_LIST_PATH, crop_size=(1024, 512), mean=IMG_MEAN, scale=False, mirror=False, set=SET),
                                    batch_size=2, shuffle=False, pin_memory=True)

In [9]:
interp = nn.Upsample(size=(1024, 2048), align_corners=False, mode='bilinear')


In [10]:
testloader_iter = iter(testloader)

In [11]:
batch = next(testloader_iter)
image, _, name = batch
output_col = ""
with torch.no_grad():
    output1 = model(Variable(image).cuda(gpu0))
    output = interp(output1).cpu().data[0].numpy()

    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)
output_col.show()

In [ ]:
image.shape

In [ ]:
output1.shape

In [ ]:
for index, batch in enumerate(testloader):
    if index % 100 == 0:
        print('%d processd' % index)
    image, _, name = batch
    output1 = model(Variable(image, volatile=True).cuda(gpu0))
    output = interp(output1).cpu().data[0].numpy()

    output = output.transpose(1,2,0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)

    output_col = colorize_mask(output)
    output = Image.fromarray(output)
    output_col
    break

In [ ]:
palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)
for i in range(zero_pad):
    palette.append(0)

def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask

def output_to_image(output):
    # input
    # ------------------
    #   G's output feature map :(c, w, h, num_classes)
    #
    #
    # output
    # ------------------
    #   output_color : PIL Image paint segmentaion color (1024, 2048)
    #
    #
    interp = nn.Upsample(size=(1024, 2048), mode='bilinear')
    output = interp(output).cpu().data[0].numpy()
    output = output.transpose(1, 2, 0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)
    output_color = colorize_mask(output)

    return output_color

In [ ]:
palette = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153, 250, 170, 30,
           220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60, 255, 0, 0, 0, 0, 142, 0, 0, 70,
           0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
zero_pad = 256 * 3 - len(palette)
for i in range(zero_pad):
    palette.append(0)

def colorize_mask(mask):
    # mask: numpy array of the mask
    new_mask = Image.fromarray(mask.astype(np.uint8)).convert('P')
    new_mask.putpalette(palette)

    return new_mask

def output_to_image(output):
    # input
    # ------------------
    #   G's output feature map :(c, w, h, num_classes)
    #
    #
    # output
    # ------------------
    #   output_color : PIL Image paint segmentaion color (1024, 2048)
    #
    #
    interp = nn.Upsample(size=(1024, 2048), mode='bilinear')
    output = interp(output).cpu().data[0].numpy()
    output = output.transpose(1, 2, 0)
    output = np.asarray(np.argmax(output, axis=2), dtype=np.uint8)
    output_color = colorize_mask(output)

    return output_color

In [19]:
def output_to_image(output):
    # input
    # ------------------
    #   G's output feature map :(c, w, h, num_classes)
    #
    #
    # output
    # ------------------
    #   output_color : PIL Image paint segmentaion color (1024, 2048)
    #
    #
    interp = nn.Upsample(size=(1024, 2048), mode='bilinear')
    output = interp(output).permute(0,2,3, 1)
    _, output = torch.max(output, -1)
    output = output.cpu().data[0].numpy().astype(np.uint8)
    output_color = colorize_mask(output)

    return output_color

In [ ]:
import torchvision.transforms as transforms

In [20]:
output_col = ""
a = 0
with torch.no_grad():
    output1 = model(Variable(image).cuda(gpu0))
    output_col = output_to_image(output1)
    output_col.show()
    

C:\Users\VIPLAB\Anaconda2\envs\xiao\lib\site-packages\torch\nn\functional.py:1749: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
